In [1]:
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By


In [2]:
driver = webdriver.Chrome()  # Mac用の記述
driver.implicitly_wait(10)


In [12]:
# 動作確認用のデータフレームを作成
df_for_export = pd.DataFrame(
    data={
        "コード": [
            "1434",
            "1518",
            "1775",
            "9603",
        ],
        "1株配当": [None, None, None, None],
    }
)
dtypes = {"コード": "object", "1株配当": "float"}
df_for_export = df_for_export.astype(dtypes)


In [ ]:
# 各銘柄のみんかぶの配当ページから1株配当金を取得する

for i, stock_code in enumerate(df_for_export["コード"]):
    # 配当ページへアクセス
    url_dividend = f"https://minkabu.jp/stock/{stock_code}/dividend"
    driver.get(url_dividend)
    time.sleep(2)
    # 予想配当金がないものは0円として処理する
    list_element = driver.find_elements(
        By.CLASS_NAME, "dividend-state__amount__integer"
    )
    if list_element == []:
        df_for_export.loc[i, "1株配当"] = 0
        print(f"{stock_code} skipped")
        continue
    # ページのhtmlを取得してパースする
    html = driver.page_source.encode("utf-8")
    parsed_html = BeautifulSoup(html, "html.parser")
    # 1株配当金を取得する
    dividend_integer = (
        parsed_html.find_all(class_="dividend-state__amount__integer")[0]
        .get_text()
        .strip()
    )  # 整数部分＋小数点
    dividend_decimal = (
        parsed_html.find_all(class_="dividend-state__amount__decimal")[0]
        .get_text()
        .strip()
    )  # 小数部分
    dividend = float(dividend_integer + dividend_decimal)
    df_for_export.loc[i, "1株配当"] = dividend
